In [31]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool

import os
import requests
import random

In [ ]:
load_dotenv()

os.environ['LANGCHAIN_PROJECT'] = 'RasoTest'

In [33]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",  # HF model
    task="text-generation",
)

llm = ChatHuggingFace(llm=llm)

In [34]:
# Tools 
search_tool = DuckDuckGoSearchRun(region="us-en")


@tool
def calculator(first_num: float, second_num: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.
    Supported operation: add, sub, mul, div
    """
    try:
        if operation == "add":
            result = first_num + second_num
        elif operation == "sub":
            result = first_num - second_num
        elif operation == "mul":
            result = first_num * second_num
        elif operation == "div":
            if second_num == 0:
                return {"error": "Division by zero is not allowed"}
            result = first_num / second_num
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        
        return {"first_num": first_num, "second_num": second_num, "operation": operation, "result": result}
    
    except Exception as e:
        return {"error": str(e)}
    

@tool
def get_stock_price(symbol: str)-> dict:
    """
    Fetch latest stock price for agiven symbol (e.g. 'AAPL', 'TSLA')
    using Alpha Vantage with API key in URL.
    """
    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=47M1Y00C8TAKM5XZ'
    r = requests.get(url)
    return  r.json()


In [35]:
# make tool list
tools = [get_stock_price, search_tool, calculator]

# make the llm tool_aware
llm_with_tools = llm.bind_tools(tools)

In [36]:
# State
class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]

In [37]:
# graph nodes
def chat_node(state: ChatState):
    """LLM node that may answer or request a tool call."""
    message = state['messages']
    response = llm_with_tools.invoke(message)
    return {"messages": [response]}

tool_node = ToolNode(tools)  # Executes tool calls

In [38]:
# graph structure
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)
graph.add_node('tools', tool_node)

graph.add_edge(START, "chat_node")
graph.add_conditional_edges("chat_node", tools_condition)
graph.add_edge("tools", "chat_node")   

In [46]:
chatbot = graph.compile()
print(chatbot.get_graph().draw_mermaid()) 

# chatbot

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chat_node(chat_node)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> chat_node;
	chat_node -.-> __end__;
	chat_node -.-> tools;
	tools --> chat_node;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [40]:
# llm.invoke("HI")

In [41]:
# regular chat
out = chatbot.invoke({'messages': [HumanMessage(content="Hello!")]})
print(out['messages'][-1].content)

Hello! How can I assist you today?


In [42]:
# Chat requiring tool
out = chatbot.invoke({'messages': [HumanMessage(content="What is 2x3")]})
print(out['messages'][-1].content)

2 × 3 = **6**


In [43]:
# Chat requiring tool
out = chatbot.invoke({'messages': [HumanMessage(content="What is the stock price of Nvidia")]})
print(out['messages'][-1].content)

The most recent price for NVIDIA (ticker **NVDA**) is **$178.19** per share (as of the latest trading day, 2025‑09‑26).


In [44]:
# Chat requiring tool
out = chatbot.invoke({'messages': [HumanMessage(content="What is the stock price of Nvidia. How much would it cost to purchase 50 shares")]})
print(out['messages'][-1].content)

**Nvidia (NVDA)**  
- Current price (latest close): **$178.19 USD** per share  
- Cost for 50 shares:  

\[
50 \text{ shares} \times \$178.19 = \$8{,}909.50
\]

So you would need approximately **$8,909.50** to purchase 50 shares of NVDA at the current price.
